In [1]:
import pandas as pd
pd.set_option('display.max_colwidth',100)

In [3]:
Data=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/Tamil/tamil_hope_first_train.csv',names=["text","label","NaN"], sep='\t')
Data.pop('NaN')
Data.head(9)

,text,label
0,Realme india product,not-Tamil
1,I also don't have tiktok hello and allnBut I'm using xiaomi,Hope_speech
2,Thalaivare..neengale inum one plus mobile vachurukinga...adha udachurunga,Non_hope_speech
3,Annee varanda thondai.. corona virus affect pannum.. neenga pathukonga. Pause koduth pasuga anne...,Hope_speech
4,5views but 18likes,Non_hope_speech
5,China phone vakathiga inu evanga ellam tiktok la video poduravagal nChina Phone vendam but Chin...,Non_hope_speech
6,Cisco webex meetings bro,Non_hope_speech
7,Yaaru madan gowriyaa ban panrenu sonnaaru? Yaarukita solringa,Non_hope_speech
8,Sabaash semma aarambam,Non_hope_speech


In [4]:
data=Data[['text','label']]
#data.head()

In [5]:
import re

def deEmojify(txt):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',txt)
Data['text']=Data['text'].apply(lambda x: deEmojify(x))

In [6]:
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)
Data['text']=Data['text'].apply(lambda x:remove_URL(x))
Data['text']=Data['text'].str.replace(r'\d+','')           #remove numbers


In [7]:
## task into array
## task into array
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
labelEncode=preprocessing.LabelEncoder()
labelEncode.fit(Data['label'])
print (labelEncode.classes_)
label=labelEncode.transform(Data['label'])
label

['Hope_speech' 'Non_hope_speech' 'not-Tamil']


array([2, 0, 1, ..., 1, 0, 0])

# tokenizer


In [8]:
#lemmatizer and tokenization   (good for noun)
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
Punctuations= "!'$%&()*+,-./:;”<=>?[\\]^_`{|}~\t\n"

wordnet_lemmatizer=WordNetLemmatizer()   #lemmatizer object
tok =nltk.tokenize.WhitespaceTokenizer() #Tokenizer object
def lemma(sentence):
    token_words=tok.tokenize(sentence)    #Tokenization
    token_words
    for word in token_words:
        if word in Punctuations:
            token_words.remove(word)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(wordnet_lemmatizer.lemmatize(word))      #Stemming
        stem_sentence.append(" ")                      #Join words
    return "".join(stem_sentence)

Data['Lemma']=Data['text'].apply(lambda x:lemma(x))
Data[['Lemma']].head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,Lemma
0,Realme india product
1,I also don't have tiktok hello and allnBut I'm using xiaomi
2,Thalaivare..neengale inum one plus mobile vachurukinga...adha udachurunga
3,Annee varanda thondai.. corona virus affect pannum.. neenga pathukonga. Pause koduth pasuga anne...
4,view but like


In [9]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf1 = TfidfVectorizer(ngram_range=(1,1),min_df =2,max_df =0.5)                #tfidf object   for 1 gramm        
features1 = tfidf1.fit_transform(Data['Lemma'])  
vector1=features1.toarray()
vector1.shape

(16160, 7450)

In [10]:
tfidf2 = TfidfVectorizer(ngram_range=(2,2),min_df =2,max_df =0.5)                #tfidf object   for 2 gramm        
features2 = tfidf2.fit_transform(Data['Lemma'])  
vector2=features2.toarray()
vector2.shape

(16160, 10263)

In [11]:
tfidf12 = TfidfVectorizer(ngram_range=(1,2),min_df =2,max_df =0.5)                #tfidf object   for 1,2 gramm        
features12 = tfidf12.fit_transform(Data['Lemma'])  
vector12=features12.toarray()
vector12[1]

array([0., 0., 0., ..., 0., 0., 0.])

In [12]:
tfidf123 = TfidfVectorizer(ngram_range=(1,3),min_df =2,max_df =0.5)                #tfidf object   for 123gramm        
features123 = tfidf123.fit_transform(Data['Lemma'])  
vector123=features123.toarray()
vector123.shape

(16160, 21716)

#spliting into training and validation

In [13]:
from sklearn.model_selection import train_test_split
#train_1,val_1,train_2,val_2,train_12,val_12,train_123,val_123,train,test =train_test_split(vector1,vector2,vector12,vector123,label,test_size=0.30,random_state=24)

In [14]:
#print(train_1.shape)
#print(train_2.shape)
#print(train_12.shape)
#print(train_123.shape)
#print(train.shape)

In [15]:
#print(val_1.shape)
#print(val_2.shape)
#print(val_12.shape)
#print(val_123.shape)
#print(test.shape)

#testing DATA

In [16]:
tDATA=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/Tamil/tamil_hope_first_dev.csv',names=["text","label","NaN"], sep='\t')
tDATA.pop('NaN')

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2013   NaN
2014   NaN
2015   NaN
2016   NaN
2017   NaN
Name: NaN, Length: 2018, dtype: float64

In [17]:
#tDATA['text']=tDATA['tweet'].apply(lambda x:remove_sw(x))
tDATA['text']=tDATA['text'].apply(lambda x: deEmojify(x))
tDATA['text']=tDATA['text'].str.replace(r'\d+','') 
#tDATA.head()
tDATA['Lemma']=tDATA['text'].apply(lambda x:lemma(x))


In [18]:
t_features1 = tfidf1.transform(tDATA['Lemma'])  
t_vector1=t_features1.toarray()
t_vector1.shape

(2018, 7450)

In [19]:
t_features2 = tfidf2.transform(tDATA['Lemma'])  
t_vector2=t_features2.toarray()
t_vector2.shape

(2018, 10263)

In [20]:
t_features12 = tfidf12.transform(tDATA['Lemma'])  
t_vector12=t_features12.toarray()
t_vector12.shape

(2018, 17713)

In [21]:
t_features123 = tfidf123.transform(tDATA['Lemma'])  
t_vector123=t_features123.toarray()
t_vector123.shape

(2018, 21716)

In [22]:
t_label=labelEncode.transform(tDATA['label'])
t_label


array([0, 2, 0, ..., 1, 2, 1])

# SVM

#for 1 gram

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.svm import SVC
model_SVM = SVC()
model_SVM.fit(vector1,label)
pred_SVM = model_SVM.predict(t_vector1)
print("SVM")
print("Accuracy score =", accuracy_score(t_label, pred_SVM))
print(metrics.classification_report(t_label,pred_SVM))

#for 2 gram

In [ ]:
model_SVM2 = SVC()
model_SVM2.fit(vector2, label)
pred_SVM = model_SVM2.predict(t_vector2)
print("SVM")
print("Accuracy score =", accuracy_score(t_label, pred_SVM))
print(metrics.classification_report(t_label,pred_SVM))

#for 12 gram

In [ ]:
model_SVM12 = SVC()
model_SVM12.fit(vector12,label)
pred_SVM = model_SVM12.predict(t_vector12)
print("SVM")
print("Accuracy score =", accuracy_score(t_label, pred_SVM))
print(metrics.classification_report(t_label,pred_SVM))

#for 123 gram

In [ ]:
model_SVM123 = SVC()
model_SVM123.fit(vector123,label)
pred_SVM = model_SVM123.predict(t_vector123)
print("SVM")
print("Accuracy score =", accuracy_score(t_label, pred_SVM))
print(metrics.classification_report(t_label,pred_SVM))

#RANDOM FOREST

In [ ]:
#for 1 gramm|

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100,max_depth=None,min_samples_split=2, random_state=0)
rf.fit(vector1,label)
pred_rf = rf.predict(t_vector1)
print("random")
print("Accuracy score =", accuracy_score(t_label, pred_rf))
print(metrics.classification_report(t_label, pred_rf))

In [ ]:
# for 2 gram
rf.fit(vector2,label)
pred_rf = rf.predict(t_vector2)
print("random")
print("Accuracy score =", accuracy_score(t_label, pred_rf))
print(metrics.classification_report(t_label, pred_rf))

In [ ]:
# for 12 gram
rf.fit(vector12,label)
pred_rf12 = rf.predict(t_vector12)
print("random")
print("Accuracy score =", accuracy_score(t_label, pred_rf12))
print(metrics.classification_report(t_label, pred_rf12))

In [ ]:
#for 123 gram
rf.fit(vector123,label)
pred_rf123 = rf.predict(t_vector123)
print("random")
print("Accuracy score =", accuracy_score(t_label, pred_rf123))
print(metrics.classification_report(t_label, pred_rf123))


#LOGESTIC REGRESSION

In [ ]:
# FOR 1 gram
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(vector1,label)
pred_LR = LR.predict(t_vector1)
print("Logistic Regression")
print("Accuracy score =", accuracy_score(t_label, pred_LR))
print(metrics.classification_report(t_label, pred_LR ))

In [ ]:
# for 2 gram

LR.fit(vector2,label)
pred_LR = LR.predict(t_vector2)
print("Logistic Regression")
print("Accuracy score =", accuracy_score(t_label, pred_LR))
print(metrics.classification_report(t_label, pred_LR ))

In [ ]:
#for 12 gram
LR.fit(vector12,label)
pred_LR12 = LR.predict(t_vector12)
print("Logistic Regression")
print("Accuracy score =", accuracy_score(t_label, pred_LR12))
print(metrics.classification_report(t_label, pred_LR12 ))

In [ ]:
# for 123 gram
LR.fit(vector123,label)
pred_LR123 = LR.predict(t_vector123)
print("Logistic Regression")
print("Accuracy score =", accuracy_score(t_label, pred_LR123))
print(metrics.classification_report(t_label, pred_LR123 ))

#KNeighbors                                        is bad

In [ ]:

#for 1 gram

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 5)
neigh.fit(vector1,label)
pred_KNN = neigh.predict(t_vector1)
print("KNN")
print("Accuracy score =", accuracy_score(t_label, pred_KNN))
print(metrics.classification_report(t_label,pred_KNN ))

In [ ]:
# for 2 gram
neigh.fit(vector2,label)
pred_KNN = neigh.predict(t_vector2)
print("KNN")
print("Accuracy score =", accuracy_score(t_label, pred_KNN))
print(metrics.classification_report(t_label,pred_KNN ))

In [ ]:
# for 12 gram
neigh.fit(vector12,label)
pred_KNN12 = neigh.predict(t_vector12)
print("KNN")
print("Accuracy score =", accuracy_score(t_label, pred_KNN12))
print(metrics.classification_report(t_label,pred_KNN12 ))

In [ ]:
# for 123 gram
neigh.fit(vector123,label)
pred_KNN123 = neigh.predict(t_vector123)
print("KNN")
print("Accuracy score =", accuracy_score(t_label, pred_KNN123))
print(metrics.classification_report(t_label,pred_KNN123 ))

# NAIVE BAYES 

In [ ]:
# for 1 gram
from sklearn.naive_bayes import GaussianNB
naive = GaussianNB()
naive.fit(vector1,label)
pred_naive = naive.predict(t_vector1)
print("Naive Bayes")
print("Accuracy score =", accuracy_score(t_label, pred_naive))
print(metrics.classification_report(t_label, pred_naive ))

In [ ]:
# for 2 gram

naive.fit(vector2,label)
pred_naive = naive.predict(t_vector2)
print("Naive Bayes")
print("Accuracy score =", accuracy_score(t_label, pred_naive))
print(metrics.classification_report(t_label, pred_naive ))

In [ ]:
# for 12 gram

naive.fit(vector12,label)
pred_naive = naive.predict(t_vector12)
print("Naive Bayes")
print("Accuracy score =", accuracy_score(t_label, pred_naive))
print(metrics.classification_report(t_label, pred_naive ))

In [ ]:
# for 123 gram

naive.fit(vector123,label)
pred_naive = naive.predict(t_vector123)
print("Naive Bayes")
print("Accuracy score =", accuracy_score(t_label, pred_naive))
print(metrics.classification_report(t_label, pred_naive ))

#GradientBoosting

In [ ]:
# for 1 gram
from sklearn.ensemble import GradientBoostingClassifier
gradient = GradientBoostingClassifier(n_estimators=100,max_depth=None,min_samples_split=2, random_state=0)
gradient.fit(vector1,label)
pred_gradient = gradient.predict(t_vector1)
print("Gradient Boosting")
print("Accuracy score =", accuracy_score(t_label,pred_gradient))
print(metrics.classification_report(t_label, pred_gradient ))

In [ ]:
# for 2 gram
gradient.fit(vector2,label)
pred_gradient = gradient.predict(t_vector2)
print("Gradient Boosting")
print("Accuracy score =", accuracy_score(t_label,pred_gradient))
print(metrics.classification_report(t_label, pred_gradient ))

In [ ]:
# for 12 gram
gradient.fit(vector12,label)
pred_gradient12 = gradient.predict(t_vector12)
print("Gradient Boosting")
print("Accuracy score =", accuracy_score(t_label,pred_gradient12))
print(metrics.classification_report(t_label, pred_gradient12 ))

In [ ]:
# for 123 gram
gradient.fit(vector123,label)
pred_gradient123 = gradient.predict(t_vector123)
print("Gradient Boosting")
print("Accuracy score =", accuracy_score(t_label,pred_gradient123))
print(metrics.classification_report(t_label, pred_gradient123 ))

#DECISION TREE

In [ ]:
# for 1 gram
from sklearn.tree import DecisionTreeClassifier
decision = DecisionTreeClassifier()
decision.fit(vector1,label)
pred_decision = decision.predict(t_vector1)
print("Decision Tree")
print("Accuracy score =", accuracy_score(t_label,pred_decision))
print(metrics.classification_report(t_label,pred_decision ))

In [ ]:
# for 2 gram
decision.fit(vector2,label)
pred_decision2 = decision.predict(t_vector2)
print("Decision Tree")
print("Accuracy score =", accuracy_score(test,pred_decision2))
print(metrics.classification_report(test,pred_decision2 ))

In [ ]:
# for 12 gram
decision.fit(vector12,label)
pred_decision12 = decision.predict(t_vector12)
print("Decision Tree")
print("Accuracy score =", accuracy_score(t_label,pred_decision12))
print(metrics.classification_report(t_label,pred_decision12 ))

In [ ]:
# for 123 gram
decision.fit(vector123,label)
pred_decision123 = decision.predict(t_vector123)
print("Decision Tree")
print("Accuracy score =", accuracy_score(t_label,pred_decision123))
print(metrics.classification_report(t_label,pred_decision123 ))